In [1]:
import numpy as np
from matplotlib import pyplot as plt
import json
from tqdm.notebook import tqdm
import sys
import corner
import matplotlib.image as mpimg

sys.path.append("../")

import tdpy

from binary_planets.sim import get_hill_radius

%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
def gen_arr(config, bin=True, n=12000, secondary=True):
    n_secondary = config["n_secondary"]
    l = bin*len(config["binary"].keys()) + secondary*n_secondary*len(config["secondary_0"].keys()) + 1
    arr = np.zeros((n, l))*np.nan
    keys = []
    keys.append("m_star")
    if secondary:
        for i in range(0, n_secondary):
            for key in config["secondary_0"].keys():
                keys.append(f"{i}/{key}")
    if bin:
        for key in config["binary"].keys():
            keys.append(f"bin/{key}")
    
    return arr, keys




def add_to_array(arr, config, keys, i, bin=True, n=12000, secondary=True):
    if not np.any(arr):
        arr, keys = gen_arr(config, bin=bin, n=n, secondary=secondary)
        
    n_secondary = config["n_secondary"]
    arr[i, 0] = config["m_star"]
    j = 1
    
    if secondary:
        for k in range(0, n_secondary):
            for key in config["secondary_0"].keys():
                arr[i, j] = config[f"secondary_{k}"][key]
                j += 1
    if bin:
        for key in config["binary"].keys():
            arr[i, j] = config["binary"][key]
            j += 1
    
    return arr, keys
    

In [3]:

run_systems = np.load("../data/compact_systems_run_composite.npy", allow_pickle=True)
names = [sys["name"] for sys in run_systems]
names = ["Kepler-150"]


dir = "no_inj"
arr_t = None
stable_t = None
for system in tqdm(names):
    try:
        arr = None
        keys = None
        system_idx = np.where(np.array(names) == system)[0][0]


        j = 0
        stable = np.zeros(1000)

        for i in range(0, 1000):
            try:
                with open(f"../output/{dir}/{system}/{i}/config.json") as f:
                    cfg = json.load(f)
                    arr, keys = add_to_array(arr, cfg, keys, j, bin=False, secondary=True)

                elements = np.load(f"../output/{dir}/{system}/{i}/elements.npy")
                a_i = elements[0, :, 0]
                a_f = elements[-1, :, 0]
                stable[i] = np.all(((a_i-a_f)/a_i) < 0.1)
                j += 1

            except Exception as e:
#                 print(e)
                pass
        arr = arr[:j, :]
        stable = stable[:j]
        stable = np.logical_not(np.logical_not(stable))
        keys = np.array(keys)
        arr += np.random.random(arr.shape) * 1e-20


        m_tot = arr[:, keys=="bin/m1"] + arr[:, keys=="bin/m2"]
        q = arr[:, keys=="bin/m1"] / m_tot
        arr[:, keys=="bin/m1"] = m_tot
        arr[:, keys=="bin/m2"] = q
        keys[keys=="bin/m1"] = "bin/m_tot"
        keys[keys=="bin/m2"] = "bin/q"

        hill_radius = get_hill_radius(arr[:, keys=="bin/a"], 
                                      arr[:, keys=="bin/e_sys"], 
                                      arr[:, keys=="bin/m_tot"], 
                                      arr[:, keys=="m_star"])
        arr[:, keys=="bin/d"] /= hill_radius

        arr[:, keys=="bin/a"] = (np.log(arr[:, keys=="bin/a"]/run_systems[system_idx]["gap"][0]) 
                                 / np.log(run_systems[system_idx]["gap"][1]/run_systems[system_idx]["gap"][0]))

        if not np.any(arr_t):
            arr_t = arr
            stable_t = stable
        else:
            arr_t = np.vstack([arr_t, arr])
            stable_t = np.hstack([stable_t, stable])
    except Exception as e:
        print(e)

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
idxs = np.array(["inc" not in key for key in keys])
# idxs = np.array([("inc" not in key) and ("/e" not in key) for key in keys])
idxs = np.array([True for key in keys])


param_labels = np.char.replace(np.array(keys)[idxs], "_", "-")



# labels of the populations
listlablpopl = ['stable', 'not stable']

# number of populations
numbpopl = len(listlablpopl)


# number of features
numbfeat = np.sum(idxs)

# generate samples


# labels of the parameters
listlablpara = []
for k in range(numbfeat):
    listlablpara.append([param_labels[k], ''])

typepgrp = 'both'
    
listcolrpopl = ["red", "gray"]
    
tdpy.plot_grid( [arr_t[stable_t, :][: ,idxs], arr_t[np.logical_not(stable_t),:][:,idxs]],
               listlablpara, listlablpopl=listlablpopl, typeplottdim='scat', 
               pathbase=f"../figs/corner2", listcolrpopl =listcolrpopl )
# plt.savefig("../figs/corner.pdf")


Number of samples in the populations:
stable
23
not stable
2


In [1]:
keys

NameError: name 'keys' is not defined

In [2]:
cfg

NameError: name 'cfg' is not defined

In [ ]:
plt.imshow(mpimg.imread('../figs/corner_KOI-351_pmar_scat_None.png'))

In [52]:
arr[stable, :][:, idxs].shape

(14, 31)

In [22]:
idxs = np.array([("inc" not in key) and ("/e" not in key) for key in keys])



param_labels = np.char.replace(np.array(keys)[idxs], "_", "-")



# labels of the populations
listlablpopl = ['stable', 'not stable']

# number of populations
numbpopl = len(listlablpopl)


# number of features
numbfeat = np.sum(idxs)

# generate samples


# labels of the parameters
listlablpara = []
for k in range(numbfeat):
    listlablpara.append([param_labels[k], ''])

typepgrp = 'both'
    
j 
tdpy.plot_grid( [arr[stable, :][: ,idxs][:, :j], arr[np.logical_not(stable),:][:,idxs][:, :j]], listlablpara[:j], listlablpopl=listlablpopl, typeplottdim='scat', pathbase="../figs/corner")
# plt.savefig("../figs/corner.pdf")
# plt.show()

Number of samples in the populations:
not stable
169
stable
831
Writing to ../figs/cornerpmar_scat_None.png...


In [26]:
id(cfg)

139879228196352

In [35]:
c = dict(cfg)
id(c)

139879441600256

In [38]:
c["name"]

'bin_inj/Kepler-102/999'

In [37]:
cfg["name"] = "test"